# Using URLs as Context with Gemini

This notebook demonstrates how to provide URLs as context to the Gemini API. The `url_context` tool allows Gemini to extract content from provided URLs as additional context for prompts. This is useful if you want to provide additional uptodate or external context to the model that is not available as part of the internal knowledge of the model.

When `url_context` is enabled the model will use all provided URLs (up to 20) to inform its response. This can be combined with `google_search` to provide a broader context before using the URLs. See below for 2 examples.

Url context is billed per additional input tokens added.

**Supported Models:**
- Gemini 2.5 Pro
- Gemini 2.5 Flash 
- Gemini 2.5 Flash-Lite 
- Gemini 2.5 Flash Live 
- Gemini 2.0 Flash Live


For more details, see the [official documentation](https://ai.google.dev/gemini-api/docs/url-context).


In [ ]:
%uv pip install google-genai --upgrade

**url context, multiple urls**

Will use the `llms.txt` for gemini by example to generate a snippet on how create an image

In [1]:
from google import genai

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents='''Create a example on how to use Gemini? Using the docs 
https://ai.google.dev/gemini-api/docs/text-generation 
https://ai.google.dev/gemini-api/docs/quickstart''',
    config={"tools": [{"url_context": {}}]},
)

# print the response
print(f"Response: {response.text}")

Response: To use the Gemini API, you'll first need to install the Google GenAI SDK and set up your API key. Then you can make requests to generate text.

Here's an example of how to get started with the Gemini API using Python:

**1. Get an API Key**
You can obtain a free Gemini API key from Google AI Studio.

**2. Install the Google GenAI SDK**
Use pip to install the Python SDK:
**3. Set your API Key**
The client automatically picks up the API key if it's set as an environment variable `GEMINI_API_KEY`. It's recommended to set it this way.

**4. Basic Text Generation**
Here's a basic example that generates text from a single text input using the `gemini-2.5-flash` model.

```python
from google import genai

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Explain how AI works in a few words"
)
print(response.text)
```

**5. Multi-turn Conversatio

In [2]:
# urls used for grounding
print(f"Context used: {', '.join([url.retrieved_url for url in response.candidates[0].url_context_metadata.url_metadata])}")

Context used: https://ai.google.dev/gemini-api/docs/text-generation, https://ai.google.dev/gemini-api/docs/quickstart


In [3]:
# citations
[response.candidates[0].grounding_metadata.grounding_supports]

[[GroundingSupport(
    grounding_chunk_indices=[
      0,
    ],
    segment=Segment(
      end_index=300,
      start_index=224,
      text="""Get an API Key**
  You can obtain a free Gemini API key from Google AI Studio."""
    )
  ),
  GroundingSupport(
    grounding_chunk_indices=[
      0,
    ],
    segment=Segment(
      end_index=536,
      start_index=500,
      text="It's recommended to set it this way."
    )
  ),
  GroundingSupport(
    grounding_chunk_indices=[
      1,
      0,
    ],
    segment=Segment(
      end_index=670,
      start_index=543,
      text="""Basic Text Generation**
  Here's a basic example that generates text from a single text input using the `gemini-2.5-flash` model."""
    )
  ),
  GroundingSupport(
    grounding_chunk_indices=[
      1,
    ],
    segment=Segment(
      end_index=1111,
      start_index=968,
      text="""Multi-turn Conversations (Chat)**
  The SDKs provide functionality to manage multi-turn conversations, keeping track of the co

## Combine with Context URL with Google Search


In [4]:
import os
from google import genai

# create client
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY","xxx"))

# Generate a list of cookie recipes
response = client.models.generate_content(
    model="gemini-2.5-flash-preview-05-20",
    contents='find the latest versions of the libraries used here and create a pip install command. https://gist.github.com/ultrafunkamsterdam/b1655b3f04893447c3802453e05ecb5e',
    config={"tools": [
        {"url_context": {}},
        {"google_search": {}},
    ]},
)

# print the response
print(f"Response: {response.text}")
# urls used for grounding
print(f"Context used: {', '.join([url.retrieved_url for url in response.candidates[0].url_context_metadata.url_metadata])}")
# searched websited 
print(f"Website used: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")

Response: The latest version of FastAPI is 0.116.1, released on July 11, 2025. The latest version of Uvicorn is 0.35.0, released on June 28, 2025.

Here is the `pip install` command with the latest versions:

`pip install fastapi==0.116.1 uvicorn==0.35.0`
Context used: https://gist.github.com/ultrafunkamsterdam/b1655b3f04893447c3802453e05ecb5e
Website used: FastAPI support for React ( with working react-router ) · GitHub, pypi.org, sourceforge.net
